# Name: Nalet Meinen¶

# Assignment 2 ATML 2020
## Classification with limited data
ImageNet is a well known dataset with 1000 image classes. We will be working on a subset of the dataset (60k images, 100 classes, 600 images per class 80$\times$80 pixels, RGB) and train a model to classify an image into one of the 100 classes. The dataset is located under the "data" directory. Training and validation data splits are under "data/train" and "data/val" directories respectively. Both splits consist of 100 directories, each representing an object category.

## Task 1. Implement ImageNetLimited class for data loading in datasets.py file

In [1]:
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop
from datasets import ImageNetLimited
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print("This nootebook ran on",device.type,"With number of GPU:",torch.cuda.device_count())

train_dir = 'data/train'
validation_dir = 'data/val'
len_classes = 100

# write your code
transforms = Compose(   [Resize(64),
                        ToTensor(),
                        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

_train_dataset = ImageFolder(train_dir, transform=transforms)
_val_dataset = ImageFolder(validation_dir, transform=transforms)

train_dataset = ImageNetLimited(_train_dataset)
val_dataset = ImageNetLimited(_val_dataset)

batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0) #set num_workers = 0, to fix BrokenPipeError in Windows
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

This nootebook ran on cuda With number of GPU: 1


## Task 2. CNN Architecture
Design and implement a Convolutional Neural Network architecture for image classification in a **ConvNet** class in the notebook. Some examples of popular classification models are: AlexNet, VGG, ResNet, ... Justify your design choices in the report. The input to your model must be an image of size $64 \times 64$ pixels.

In [2]:
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, num_classes=100):
        super(ConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Task 3. Train Model
Implement training and evaluation code for your model. Choose an appropriate loss function and evaluate the model on the validation set using classification accuracy. You are not allowed to use a pre-trained model (must train from scratch on the provided data).<br>
<font color='red'>Your model should achieve an accuracy of at least 40.0% on the validation set (Model with performance smaller than 40.0% will result in 0 points for this task).</font><br>

In [3]:
from utils_train import fit
model_conv = ConvNet()
model_conv = model_conv.to(device)
learning_rate = 0.01
optimizer = torch.optim.Adam(model_conv.parameters(), lr=learning_rate)
n_epochs = 10
loss_fn = nn.CrossEntropyLoss()
curves_conv2 = fit(train_dataloader, val_dataloader, model_conv, optimizer, loss_fn, n_epochs)

Training iteration 0: loss 4.6054
Training iteration 100: loss 4.5961
Training iteration 200: loss 4.6258
Training iteration 300: loss 4.5815
Training iteration 400: loss 4.5969
Training iteration 500: loss 4.6186
Training iteration 600: loss 4.6150
Training iteration 700: loss 4.6092
Training iteration 800: loss 4.6048
Training iteration 900: loss 4.6054
Training iteration 1000: loss 4.6072
Training iteration 1100: loss 4.6051
Training iteration 1200: loss 4.6061
Training iteration 1300: loss 4.6166
Training iteration 1400: loss 4.5993
Training iteration 1500: loss 4.5983
Training iteration 1600: loss 4.5960
Training iteration 1700: loss 4.6075
Training iteration 1800: loss 4.6125

Test average loss: 4.6085, accuracy: 1.005
Epoch 1/10: train_loss: 1064.9214, train_accuracy: 0.9717, val_loss: 4.6085, val_accuracy: 1.0049


Training iteration 0: loss 4.6334
Training iteration 100: loss 4.6146
Training iteration 200: loss 4.6106
Training iteration 300: loss 4.6097
Training iteration 400: loss 4.6083
Training iteration 500: loss 4.6084
Training iteration 600: loss 4.6128
Training iteration 700: loss 4.6218
Training iteration 800: loss 4.5819
Training iteration 900: loss 4.6251
Training iteration 1000: loss 4.5851
Training iteration 1100: loss 4.6110
Training iteration 1200: loss 4.6215
Training iteration 1300: loss 4.5924
Training iteration 1400: loss 4.6139
Training iteration 1500: loss 4.6105
Training iteration 1600: loss 4.6494
Training iteration 1700: loss 4.6181
Training iteration 1800: loss 4.6074

Test average loss: 4.6086, accuracy: 1.005
Epoch 2/10: train_loss: 4.6103, train_accuracy: 0.9750, val_loss: 4.6086, val_accuracy: 1.0049


Training iteration 0: loss 4.6051
Training iteration 100: loss 4.5982
Training iteration 200: loss 4.6090
Training iteration 300: loss 4.5934
Training iteration 400: loss 4.6119
Training iteration 500: loss 4.6352
Training iteration 600: loss 4.6045
Training iteration 700: loss 4.6728
Training iteration 800: loss 4.6284
Training iteration 900: loss 4.6154
Training iteration 1000: loss 4.5729
Training iteration 1100: loss 4.6107
Training iteration 1200: loss 4.6096
Training iteration 1300: loss 4.6255
Training iteration 1400: loss 4.5981
Training iteration 1500: loss 4.6120
Training iteration 1600: loss 4.6105
Training iteration 1700: loss 4.6170
Training iteration 1800: loss 4.6186

Test average loss: 4.6475, accuracy: 1.005
Epoch 3/10: train_loss: 4.6107, train_accuracy: 1.0083, val_loss: 4.6475, val_accuracy: 1.0049


Training iteration 0: loss 4.6119
Training iteration 100: loss 4.6267
Training iteration 200: loss 4.5995
Training iteration 300: loss 4.6192
Training iteration 400: loss 4.6071
Training iteration 500: loss 4.6090
Training iteration 600: loss 4.5888
Training iteration 700: loss 4.6105
Training iteration 800: loss 4.6256
Training iteration 900: loss 4.6033
Training iteration 1000: loss 4.6210
Training iteration 1100: loss 4.6127
Training iteration 1200: loss 4.5951
Training iteration 1300: loss 4.6249
Training iteration 1400: loss 4.5811
Training iteration 1500: loss 4.6148
Training iteration 1600: loss 4.6015
Training iteration 1700: loss 4.5974
Training iteration 1800: loss 4.6101

Test average loss: 4.6089, accuracy: 1.005
Epoch 4/10: train_loss: 4.6105, train_accuracy: 0.9133, val_loss: 4.6089, val_accuracy: 1.0049


Training iteration 0: loss 4.6115
Training iteration 100: loss 4.5891
Training iteration 200: loss 4.5860
Training iteration 300: loss 4.6188
Training iteration 400: loss 4.6242
Training iteration 500: loss 4.5970
Training iteration 600: loss 4.6440
Training iteration 700: loss 4.6323
Training iteration 800: loss 4.6140
Training iteration 900: loss 4.6234
Training iteration 1000: loss 4.6168
Training iteration 1100: loss 4.6300
Training iteration 1200: loss 4.6087
Training iteration 1300: loss 4.6410
Training iteration 1400: loss 4.5922
Training iteration 1500: loss 4.6175
Training iteration 1600: loss 4.6215
Training iteration 1700: loss 4.5916
Training iteration 1800: loss 4.6040

Test average loss: 4.6078, accuracy: 1.005
Epoch 5/10: train_loss: 4.6105, train_accuracy: 0.8933, val_loss: 4.6078, val_accuracy: 1.0049


Training iteration 0: loss 4.6239
Training iteration 100: loss 4.6356
Training iteration 200: loss 4.6015
Training iteration 300: loss 4.6072
Training iteration 400: loss 4.6243
Training iteration 500: loss 4.6065
Training iteration 600: loss 4.6126
Training iteration 700: loss 4.6374
Training iteration 800: loss 4.6266
Training iteration 900: loss 4.6277
Training iteration 1000: loss 4.6167
Training iteration 1100: loss 4.5887
Training iteration 1200: loss 4.6005
Training iteration 1300: loss 4.5959
Training iteration 1400: loss 4.5718
Training iteration 1500: loss 4.6084
Training iteration 1600: loss 4.5856
Training iteration 1700: loss 4.6196
Training iteration 1800: loss 4.5971

Test average loss: 4.6078, accuracy: 1.005
Epoch 6/10: train_loss: 4.6105, train_accuracy: 0.9683, val_loss: 4.6078, val_accuracy: 1.0049


Training iteration 0: loss 4.6197
Training iteration 100: loss 4.6115
Training iteration 200: loss 4.5961
Training iteration 300: loss 4.6128
Training iteration 400: loss 4.5888
Training iteration 500: loss 4.6273
Training iteration 600: loss 4.6190
Training iteration 700: loss 4.6113
Training iteration 800: loss 4.5978
Training iteration 900: loss 4.6054
Training iteration 1000: loss 4.6136
Training iteration 1100: loss 4.6049
Training iteration 1200: loss 4.6157
Training iteration 1300: loss 4.6266
Training iteration 1400: loss 4.6507
Training iteration 1500: loss 4.6137
Training iteration 1600: loss 4.6003
Training iteration 1700: loss 4.6254
Training iteration 1800: loss 4.6049

Test average loss: 4.6079, accuracy: 1.005
Epoch 7/10: train_loss: 4.6104, train_accuracy: 0.9817, val_loss: 4.6079, val_accuracy: 1.0049


Training iteration 0: loss 4.5958
Training iteration 100: loss 4.5910
Training iteration 200: loss 4.6115
Training iteration 300: loss 4.6064
Training iteration 400: loss 4.6155
Training iteration 500: loss 4.5861
Training iteration 600: loss 4.6114
Training iteration 700: loss 4.6030
Training iteration 800: loss 4.6088
Training iteration 900: loss 4.5938
Training iteration 1000: loss 4.6266
Training iteration 1100: loss 4.6018
Training iteration 1200: loss 4.6015
Training iteration 1300: loss 4.6251
Training iteration 1400: loss 4.6150
Training iteration 1500: loss 4.5709
Training iteration 1600: loss 4.6149
Training iteration 1700: loss 4.6190
Training iteration 1800: loss 4.6179

Test average loss: 4.6088, accuracy: 1.005
Epoch 8/10: train_loss: 4.6103, train_accuracy: 0.9650, val_loss: 4.6088, val_accuracy: 1.0049


Training iteration 0: loss 4.6028
Training iteration 100: loss 4.5909
Training iteration 200: loss 4.5867
Training iteration 300: loss 4.5758
Training iteration 400: loss 4.5958
Training iteration 500: loss 4.5875
Training iteration 600: loss 4.6312
Training iteration 700: loss 4.6053
Training iteration 800: loss 4.6143
Training iteration 900: loss 4.6117
Training iteration 1000: loss 4.6372
Training iteration 1100: loss 4.5829
Training iteration 1200: loss 4.5881
Training iteration 1300: loss 4.6121
Training iteration 1400: loss 4.6332
Training iteration 1500: loss 4.6044
Training iteration 1600: loss 4.5927
Training iteration 1700: loss 4.6169
Training iteration 1800: loss 4.6019

Test average loss: 4.6097, accuracy: 1.005
Epoch 9/10: train_loss: 4.6101, train_accuracy: 0.9417, val_loss: 4.6097, val_accuracy: 1.0049


Training iteration 0: loss 4.6200
Training iteration 100: loss 4.6094
Training iteration 200: loss 4.5928
Training iteration 300: loss 4.6036
Training iteration 400: loss 4.6332
Training iteration 500: loss 4.6105
Training iteration 600: loss 4.5995
Training iteration 700: loss 4.6141
Training iteration 800: loss 4.6177
Training iteration 900: loss 4.6151
Training iteration 1000: loss 4.6166
Training iteration 1100: loss 4.6293
Training iteration 1200: loss 4.6241
Training iteration 1300: loss 4.6039
Training iteration 1400: loss 4.6190
Training iteration 1500: loss 4.6056
Training iteration 1600: loss 4.6058
Training iteration 1700: loss 4.6035
Training iteration 1800: loss 4.5991

Test average loss: 4.6102, accuracy: 1.005
Epoch 10/10: train_loss: 4.6326, train_accuracy: 0.9800, val_loss: 4.6102, val_accuracy: 1.0049



## Task 4. Ablations
Try to find the best performing model by tuning the model design and hyper-parameters on the validation set. Perform ablation experiments to illustrate the effect of the most important hyper-parameters. Some examples of ablations: training parameters (e.g., optimizer, learning rates, batch size), network architecture (e.g., number of layers, number of units, activation function, normalization layers), model regularization (e.g., data augmentation, dropout, weight decay, early stopping), test-time augmentation, etc...  <br>**Perform at least 5 ablations and report the performance of each on the validation set.**

In [4]:
# write your code

## Task 5. Model Errors
Evaluate the trained model on the validation set and plot 10 random mistakes that your model made.

In [5]:
# write your code

## Task 6. Competition time!
Read the images from "data/test" folder. There are no labels for these images. Run your best model on these images and save the image IDs (names) and predicted label in a file LastName.csv. You will receive a link via email to upload the CSV file to  an online system which will give you the score of your model on the held-out test set. Top 5 students with at least 40% classification accuracy will obtain bonus points.

In [6]:
# write your code